# A Reduction Tale

> Objectives:
> * Compare operations taking place in different data containers
> * Compare sizes for these data containers
> * Help deciding when it is best to use a container or another

Let's suppose that we are going to need reductions a lot and we want to choose the best container for performing them.  First, let's start by activating our MemWatcher agent:

In [1]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

In [1] used 0.020 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 30.113 MiB


and choose a different container for the data that we want to reduce, starting with a list:

## Regular lists

In [2]:
a = [float(i) for i in range(1000*1000)]

In [2] used 54.469 MiB RAM in 0.180s, peaked 0.000 MiB above current, total RAM usage 84.582 MiB


Now, proceed with a simple reduction (sum):

In [3]:
t = %timeit -o sum(a)

100 loops, best of 3: 6.13 ms per loop
In [3] used 0.059 MiB RAM in 2.548s, peaked 0.000 MiB above current, total RAM usage 84.641 MiB


which, in MFLOPS (Mega-FloatingPointOps-Per-Second) is:

In [4]:
print("MFLOPS:", round((1e6 / t.best / 1e6), 1))

('MFLOPS:', 163.1)
In [4] used 0.016 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 84.656 MiB


Ok, so that seems fast, but we don't have other references to compare with.  In addition, a list is not the best kind of container in terms of space consumption.  So let's try now a container that seems quite optimal in terms of memory savings.

## Containers using the array module in Python

In [5]:
# Create an array of 'd'oubles
import array
%load_ext memory_profiler

In [5] used 0.004 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 84.660 MiB


In [6]:
%memit arr = array.array('d', a)

peak memory: 84.86 MiB, increment: 0.19 MiB
In [6] used 0.199 MiB RAM in 0.237s, peaked 0.000 MiB above current, total RAM usage 84.859 MiB


7.7 ~ 15 MB vs 31 MB seems like a good deal, although sometimes Python is allocating more memory than necessary.  In fact, the array module seems to provide optimal containers from a memory consumption point of view:

In [7]:
# Size per element:
(mw.memory_delta * 2**20) / 1e6

0.208896

In [7] used 0.059 MiB RAM in 0.006s, peaked 0.000 MiB above current, total RAM usage 84.918 MiB


But how it performs during reductions?

In [8]:
t = %timeit -o sum(arr)

100 loops, best of 3: 8.09 ms per loop
In [8] used 0.012 MiB RAM in 3.346s, peaked 0.000 MiB above current, total RAM usage 84.930 MiB


In [9]:
print("MFLOPS:", round(1e6 / t.best / 1e6, 1))

('MFLOPS:', 123.6)
In [9] used 0.008 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 84.938 MiB


Well, that's a bit disappointing, as the array object performs about 2x slower than a regular list.  Not sure about the resons, but probably the array module is not getting too much attention performance-wise mainly because the NumPy existance.  Speaking of NumPy: here we go!

## NumPy

In [10]:
import numpy as np

In [10] used 4.648 MiB RAM in 0.039s, peaked 0.000 MiB above current, total RAM usage 89.586 MiB


In [11]:
na = np.array(a, dtype=np.float64)

In [11] used 0.043 MiB RAM in 0.028s, peaked 0.000 MiB above current, total RAM usage 89.629 MiB


We see that, with 8 bytes/element, NumPy is also an efficient container (much more than the `array` package in standard library which takes ~16 bytes/element).

In [12]:
t = %timeit -o sum(na)

10 loops, best of 3: 71.6 ms per loop
In [12] used 0.020 MiB RAM in 2.933s, peaked 0.000 MiB above current, total RAM usage 89.648 MiB


In [13]:
print("MFLOPS:", round(1e6 / t.best / 1e6, 3))

('MFLOPS:', 13.964)
In [13] used 0.004 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 89.652 MiB


Oops, this is more than several times slower than the `array` module.  Why so?

**Answer:** NumPy has a lot of overhead in producing a Python integer for every element in the array.

*Hint:* Use internal methods (ufuncs) when possible.

In [14]:
t = %timeit -o na.sum()

1000 loops, best of 3: 426 µs per loop
In [14] used 0.250 MiB RAM in 1.767s, peaked 0.000 MiB above current, total RAM usage 89.902 MiB


In [15]:
print("FLOPS:", round(1e6 / t.best / 1e6, 3))

('FLOPS:', 2345.387)
In [15] used 0.000 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 89.902 MiB


This is about 100x the speed of sum() on a Python list and it is also pretty optimal in terms of both execution time and space consumed.

## Exercise

The speed in the above reduction is limited by memory speed, not CPU speed.  Could you provide a hint on the maximum memory speed that supports your laptop?

#### Solution

In [16]:
# This is an easy one.  Just divide the size of the dataset by the time that takes the reduction
(na.size * na.itemsize) / t.best / 2**30

17.47449143638083

In [16] used 0.000 MiB RAM in 0.004s, peaked 0.000 MiB above current, total RAM usage 89.902 MiB


So, in this case the memory bandwidth is close to 18 GB/s.

## Using compressed in-memory containers with bcolz

But let us suppose that we have really big data to process in our laptop and want to see if we can store our data in less space.  Enter compression:

In [17]:
import bcolz
bcolz.print_versions()
bcolz.defaults.cparams['cname'] = 'blosclz'
bcolz.defaults.cparams['clevel'] = 9
bcolz.set_nthreads(4)

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
bcolz version:     0.12.1.dev2
NumPy version:     1.10.1
Blosc version:     1.4.1 ($Date:: 2014-07-08 #$)
Blosc compressors: ['blosclz', 'lz4', 'lz4hc', 'snappy', 'zlib']
Numexpr version:   2.4.4
Python version:    2.7.10 |Continuum Analytics, Inc.| (default, Oct 19 2015, 18:04:42) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Platform:          linux2-x86_64
Byte-ordering:     little
Detected cores:    8
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


8

In [17] used 24.461 MiB RAM in 0.129s, peaked 0.000 MiB above current, total RAM usage 114.363 MiB


In [18]:
ca = bcolz.carray(na)

In [18] used 0.809 MiB RAM in 0.007s, peaked 0.000 MiB above current, total RAM usage 115.172 MiB


In [19]:
print("mem_used:", mw.measurements.memory_delta)

('mem_used:', 0.80859375)
In [19] used 0.016 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 115.188 MiB


Ok, this time the amount of memory used seems much lower.  Also, bcolz containers can provide an estimation on how much memory they are taking; let's have a look:

In [20]:
ca

carray((1000000,), float64)
  nbytes: 7.63 MB; cbytes: 332.78 KB; ratio: 23.48
  cparams := cparams(clevel=9, shuffle=True, cname='blosclz')
[  0.00000000e+00   1.00000000e+00   2.00000000e+00 ...,   9.99997000e+05
   9.99998000e+05   9.99999000e+05]

In [20] used 0.559 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 115.746 MiB


In this case we see that bcolz estimation is reasonably close to `ipython_memwatcher` measurements.  Let's have a look at the speed of the reduction:

In [21]:
t = %timeit -o ca.sum()
print("MFLOPS:", round(1e6 / t.best / 1e6, 3))

1000 loops, best of 3: 1.32 ms per loop
('MFLOPS:', 759.465)
In [21] used 0.277 MiB RAM in 5.431s, peaked 0.000 MiB above current, total RAM usage 116.023 MiB


This is around 2~4x slower (depending on the machine) than a regular NumPy array, but the size of the array is an impressive 10x smaller.  But is compression the only responsible of the overhead?  Let's investigate a bit further.

## Using uncompressed containers with bcolz

In order to see if this is because of the compression overhead, let's use an uncompressed array:

In [22]:
cau = bcolz.carray(a, cparams=bcolz.cparams(clevel=0))

In [22] used 13.816 MiB RAM in 0.038s, peaked 0.000 MiB above current, total RAM usage 129.840 MiB


In [23]:
cau

carray((1000000,), float64)
  nbytes: 7.63 MB; cbytes: 7.75 MB; ratio: 0.98
  cparams := cparams(clevel=0, shuffle=True, cname='blosclz')
[  0.00000000e+00   1.00000000e+00   2.00000000e+00 ...,   9.99997000e+05
   9.99998000e+05   9.99999000e+05]

In [23] used 0.012 MiB RAM in 0.004s, peaked 0.000 MiB above current, total RAM usage 129.852 MiB


In [24]:
t = %timeit -o cau.sum()
print("MFLOPS:", round(1e6 / t.best / 1e6, 3))

1000 loops, best of 3: 1.46 ms per loop
('MFLOPS:', 682.943)
In [24] used 0.020 MiB RAM in 6.072s, peaked 0.000 MiB above current, total RAM usage 129.871 MiB


As we can see, the times with an uncompressed `carray` are very close to a compressed one (the later is actually faster here!), so compressing is not the only source of the overhead (and it is very minor in fact).  The actual source of the difference is the memory layout of the different containers (bcolz layout is a bit more complex than NumPy).

So, bcolz allows to use compressed in-memory data containers at the cost of some performance (compared with NumPy).  But the performance overhead is rather small, and sometimes you prefer to keep more data in-memory.

On another hand, in the next notebooks we are going to see that bcolz can be pretty close to NumPy, performance wise, in some scenarios.

### Exercise: Using bcolz in real scenarios

bcolz does not get good compression ratios only with synthetic data, but with real data too.  Be sure to check out this URL:

http://nbviewer.ipython.org/gist/alimanfoo/e93a532eb6bde311ea39/genotype_bitshuffle.ipynb

and let's discuss this specific case of bcolz usage in genomics:

* Which are the typical compression ratios for this case?

* Is there a difference in speed accessing data in compressed and non-compressed state (clevel=0)

* Which are the compressors achieving the best compression/speed ratio?